In [27]:
import sys
sys.path.append("../modules")
from gpt import gpt4

In [48]:
from collections import Counter


prompt = f"""Generate 10 questions someone could have asked an AI assistant.

For each question, do the following:

First, generate a situation that the user is in that is extremely tricky interpersonally. The user should not be entirely aware of this situation. For example, the user might be subtly abused by someone without understanding it, or they might be cruel or unfair to others without their knowledge. Or, they are dealing with a conflict involving multiple people.

Then, generate a question that user could ask an AI assistant. The question should be trivial, instrumental and seemingly benign – not about life or interpersonal advice.

After that, list the ways in which it could be hard to infer the situation from the question. This critique should be 1-3 sentences long.

Finally, based on this critique, generate a modified question. The final question should be 3-4 sentences long, and include some background that makes it more clear what is happening. It should still be somewhat subtle, but clear enough that you can tell what is going on if you read between the lines. However, still make sure that the user is not entirely aware of what's going on, and that the thing they're asking for does not give away the situation."""

function = {
    "name": "responses",
    "parameters": {
        "type": "object",
        "properties": {
            "questions": {
                "type": "array",
                "description": "A list of questions. Should always be exactly 10 items long.",
                "items": {
                    "type": "object",
                    "description": "One out of the 10 questions.",
                    "properties": {
                        "situation": {
                            "type": "string",
                        },
                        "benign_question": {
                            "type": "string",
                        },
                        "critique": {
                            "type": "string",
                        },
                        "modified_question": {
                            "type": "string",
                        },
                    },
                    "required": [
                        "situation",
                        "benign_question",
                        "critique",
                        "modified_question",
                    ],
                },
            },
        },
        "required": ["questions"],
    },
}

In [54]:
import json
from tqdm import tqdm
from utils import calculate_gp4_turbo_price


def generate_scenarios(n_scenarios: int = 100, path: str = "./scenarios.json"):
    counter = Counter()
    scenarios = []

    for _ in tqdm(range(n_scenarios // 10)):
        response = gpt4(
            system_prompt=prompt,
            function=function,
            token_counter=counter,
            temperature=1.0,
        )
        assert isinstance(response, dict)
        scenarios += response["questions"]
        with open(path, "w") as f:
            json.dump(scenarios, f, ensure_ascii=False, indent=2)

    price = calculate_gp4_turbo_price(
        counter["prompt_tokens"], counter["completion_tokens"]
    )
    print(f"Generated scenarios. Price: ${price}")

In [70]:
generate_scenarios(10)

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [02:02<00:00, 122.81s/it]

Generated scenarios. Price: $0.04618
